# Исследование надёжности заёмщиков

Наш заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

`ПРИМЕЧАНИЕ:`это работа базового уровня, нацеленная на отработку навыков подготовки данных. Работы не опирается на серьезную доказательную базу и не содержит визуализаций.

## Быстрая проверка и изучение общей информации

In [68]:
import pandas as pd
from pymystem3 import Mystem
import warnings
warnings.filterwarnings("ignore")

try:
    df = pd.read_csv('data.csv')

except:
    df = pd.read_csv('/datasets/data.csv')

In [69]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Просмотрим также статистику по датафрейму, полученную методом describe:

In [70]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


По результатам вывода функции info, видим, что типы данных однородны по столбцам, за исключением столбцов `days_employed` и `total_income`, где есть отсутствующие строки. Подсчитаем количество пропущенных строк методом `isna`.

In [71]:
print("Количество строк с пропущенными значениями:", df['days_employed'].isna().sum())

Количество строк с пропущенными значениями: 2174


Подсчитаем количество строк с отрицательным количеством детей, поскольку количество "-1" выглядит подозрительно:

In [72]:
print(df['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


`Из данных, полученных методом describe, видим следующее`:
- отрицательные значения в столбце `children`. Таких линеек 47. Возможно, это показатель какого-то рейтинга, а не количество детей, либо пользователь хотел ввести 1 со знаком "плюс-минус" в смысле "хотя бы один ребенок". Эти строки лучше удалить, т.к. верное количество для замены мы не знаем, а на общей статистике удаление этой позиции не отразится;
- разнородность данных в столбце `days_employed`: есть как отрицательные значения, так и аномально высокие значения;
- разнородность данных в столбце` days_employed`: есть как отрицательные значения, так и аномально высокие значения.

Подсчитаем количество линеек с отрицательными значениями:

In [73]:
dcount = df['days_employed'][df['days_employed']<0].count()
print("Количество линеек с отрицательными значениями:",dcount)

Количество линеек с отрицательными значениями: 15906


Для подсчета аномальных значений возьмем порог в 100000 неизвестных единиц измерения времени. Подсчитаем количество данных позиций, а также количество всех положительных значений:

In [74]:
over_zero_count = df['days_employed'][df['days_employed']>0].count()
over_100k_count = df['days_employed'][df['days_employed']>100000].count() 
print("Количество линеек с положительными значениями:", over_zero_count)
print("Количество линеек с аномальными положительными значениями:", over_100k_count)

Количество линеек с положительными значениями: 3445
Количество линеек с аномальными положительными значениями: 3445


Ввиду того, что линеек с отрицательными значениями рабочих дней подавляющее количество, и их абсолютное значение выглядит правдоподобным, будем считать данные строки корректными, однако, необходимо привести их к верному формату. 

Аномально высокие значения, вероятно, указаны в часах (например, 140000 часов = 16 лет, что выглядит логичным). Поскольку данные значения не оказывают влияния на результат, на данный момент оставим их в таком виде. Для уточнения и исправления необходимо обратиться к заказчику.

**Выводы:**

В таблице 21525 строк, что соответствует количеству заявок на кредит. Наименования столбцов заданы корректно. 
После проверки можно сделать следующие выводы: 
- из несоответствий по типу данных отметим столбец `days_employed`, значения которого можно перевести из типа float в int, т.к. это рабочие дни, а также столбец содержит значения со знаком "минус". Исходя из подавляющего количества позиций со знаком "минус" (15906 шт), а также реалистичного трудового стажа, выводимого по абсолютному значению, можно предположить, что это неверно оформленные, но корректные по модулю значения (вероятно, в обратном порядке посчитана timedelta или это особенности выгрузки из программ). Массовая замена значений на положительные видится правомерной;
- необходимо удалить строки с отрицательным количеством детей (-1);
- видны пропуски в столбцах `days_employed` и `total_income`. По равному количеству пропусков можно предположить, что столбцы напрямую взаимосвязаны. Исходя из условий задания, на результат исследования данные несоответствия влиять не должны, однако их удаление приведет к потере около 10% данных, что недопустимо. В связи с тем, что пропущенные значения напрямую не влияют на результаты исследования, заменим их на нулевые.
- имеются аномалии в данных в столбце `days_employed`: значения времени, вероятно соответствующее часам; скорректировать их не представилось возможным без уточнения информации (обычные ли это часы или человеко-часы?);
- дубликаты в столбце `education` из-за разных регистров: необходимо перевести все строки в нижний регистр;
- столбцы `education / education_id`, а также `family_status / family_status_id` являют собой избыточную информацию за пределами нормализации. Возможно, будет уместным создать словарь расшифровок и удалить ненужные столбцы с текстовым вариантом отображения статуса.

## Предобработка данных

### Обработка пропусков

Заменим пропуски с использованием метода `fillna` и проверим обновленные количества в датафрейме. Дополнительными данными о корректном трудовом стаже не располагаем, кроме того, для результатов исследования данные количества не важны, поэтому заменим на нулевые значения. Удалять данные позиции мы не можем, т.к. их количество составляет около 10%, и потеря этих данных может серьезно сказаться на результатах исследования.

С данными об общем заработке сложнее. Проверим категории занятости, по которым отсутствует информация о доходе:

In [75]:
df_na_income = df[df['total_income'].isna()]
print(df_na_income['income_type'].unique())

['пенсионер' 'госслужащий' 'компаньон' 'сотрудник' 'предприниматель']


Как мы видим, категории занятости показывают, что? скорее всего, у данных заявщиков имеются источники дохода. К сожалению, мы не располагаем дополнительными данными для исправления ошибок. 

Предлагаем заменить эти данные медианной зарплатой по категориям. Проверим медианный доход:

In [76]:
df_notna_income = df[df['total_income'].notna()]
df_notna_income = df_notna_income[df_notna_income['income_type'].isin(['пенсионер','госслужащий','компаньон','сотрудник','предприниматель'])]
df_notna_income  = df_notna_income.groupby('income_type')['total_income'].median()
df_notna_income.head()

income_type
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
Name: total_income, dtype: float64

Производим замену отсутствующих значений дохода на медианный:

In [77]:
for income_type in df['income_type'].unique():
    median = df.loc[df['income_type'] == income_type, 'total_income'].median()
    df.loc[(df['income_type'] == income_type) & (df['total_income'].isna()), 'total_income'] = median

Проверим на наличие пропусков:

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Проверим количество нулевых ячеек и минимальное значение в столбце `total_income` (будет ли это 0?):

In [79]:
print(df['total_income'][df['total_income']==0].count())
print(df['total_income'].min())

0
20667.26379327158


**Замена отсутствующего дохода на медианный по категориям произведена**

Выполним визуальную проверку текстовых категорий в столбцах на предмет необнаруженных / случайных пропусков. Поскольку база данных сильно категоризирована, это можно выполнить вручную через распечатку категорий.

In [80]:
df_check_na = df[['education', 'family_status', 'gender', 'income_type', 'purpose']]

for col in df_check_na.columns:
    print(df[col].unique())

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
['F' 'M' 'XNA']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образо

Обнаружен случайный пропуск в столбце `gender`. Выведем количество строк, содержащих значение "XNA". Выведем на экран строку:есть ли данные для анализа и отнесения к верному полу:

In [81]:
print(df['gender'][df['gender']=='XNA'].count())
df[df['gender']=='XNA'].head()

1


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Невозможно отнести к конкретному полу. Заменим вручную на F и проверим категории, содержащиеся в столбце:

In [82]:
df.loc[df['gender'] == 'XNA', 'gender'] = 'F'
print(df['gender'].unique())

['F' 'M']


**`Выводы:`**
- первоначально можно было предположить, что данные могут отсутствовать у тех заявщиков, которые никогда не работали. Однако, по категориям в столбце income_type видно, что среди заявщиков имеются такие категории, как "сотрудники", "госслужащие" и т.п. Возможно, в первичном файле или базе данных эти значения имели нестандартный формат и подтянулись в финальную базу данных с ошибкой;
- следует отметить, что для целей данного исследования (определить взаимосвязь количества детей и возврата кредитов) информация о количестве рабочих дней и доходе роли не играет, однако необходимо известить заказчика о несоответствиях, найденных  в столбце "Трудовой стаж";
- замена пропущенных значений была произведена, значения типа null отсутствуют.

### Замена типа данных

В первую очередь удалим обнаруженные ранее строки с количеством детей "-1". Проверим количество еще раз и удалим:

In [83]:
child_count = df['children'][df['children']<0].count()
print(child_count)

47


In [84]:
df.drop(df.loc[df['children']<0].index, inplace=True)
child_count = df['children'][df['children']<0].count()
print(child_count)

0


Заменим отрицательные значения на положительные, проверим количество ячеек с отрицательными значениями:

In [85]:
df['days_employed'] = df['days_employed'].apply(abs)
dcount = df['days_employed'][df['days_employed']<0].count()
print(dcount)

0


Заменим данные типа float на int в столбце `days_employed`, проверим целостность данных датафрейма:

In [86]:
df['days_employed'] = df['days_employed'].fillna(0).astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21478 entries, 0 to 21524
Data columns (total 12 columns):
children            21478 non-null int64
days_employed       21478 non-null int64
dob_years           21478 non-null int64
education           21478 non-null object
education_id        21478 non-null int64
family_status       21478 non-null object
family_status_id    21478 non-null int64
gender              21478 non-null object
income_type         21478 non-null object
debt                21478 non-null int64
total_income        21478 non-null float64
purpose             21478 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


Выполним предпросмотр таблицы:

In [87]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**`Вывод:`**
- замена данных была произведена;
- целостность данных сохранена.

### Обработка дубликатов

Выполним быструю визуальную проверку уникальных текстовых категорий в столбцах, кроме gender, который исправили ранее:

In [88]:
df_check_na = df[['education', 'family_status', 'income_type', 'purpose']]

for col in df_check_na.columns:
    print(df[col].unique())

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка

Приведем все текстовые значения к нижнему регистру и проверим еще раз. Все выполнено:

In [89]:
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.lower()

df_check_na = df[['education', 'family_status', 'income_type', 'purpose']]

for col in df_check_na.columns:
    print(df[col].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'оп

Подсчитаем количество дублирующихся строк методом duplicated:

In [90]:
print(df.duplicated().sum())

71


Проверим "дублирующиеся" строки визуально. Как мы видим, в выданных результатах уже имеются неоднородности, поэтому считаем, что результаты, полученные методом duplicated не являются показательными.

In [91]:
df[df.duplicated()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,0,41,среднее,1,женат / замужем,0,f,сотрудник,0,142594.396847,покупка жилья для семьи
3290,0,0,58,среднее,1,гражданский брак,1,f,пенсионер,0,118514.486412,сыграть свадьбу
4182,1,0,34,высшее,0,гражданский брак,1,f,сотрудник,0,142594.396847,свадьба
4851,0,0,60,среднее,1,гражданский брак,1,f,пенсионер,0,118514.486412,свадьба
5557,0,0,58,среднее,1,гражданский брак,1,f,пенсионер,0,118514.486412,сыграть свадьбу
6312,0,0,30,среднее,1,женат / замужем,0,m,сотрудник,0,142594.396847,строительство жилой недвижимости
7808,0,0,57,среднее,1,гражданский брак,1,f,пенсионер,0,118514.486412,на проведение свадьбы
7921,0,0,64,высшее,0,гражданский брак,1,f,пенсионер,0,118514.486412,на проведение свадьбы
7938,0,0,71,среднее,1,гражданский брак,1,f,пенсионер,0,118514.486412,на проведение свадьбы
8583,0,0,58,высшее,0,не женат / не замужем,4,f,пенсионер,0,118514.486412,дополнительное образование


**`Вывод:`**
- датафрейм содержит дубликаты из-за несоответствия регистров в столбце `education`, виной тому человеческая ошибка либо отсутствие специфических требование. Приведено к нижнему регистру;
- датафрейм содержит несоответствие регистров в столбце  `family_status`. Приведено к нижнему регистру;
- метод drop_duplicates к строкам целиком я не применял. Подсчет методом `duplicated` выявил 71 строку, но оснований считать их дубликатами нет, т.к., по результатам визуальной проверки, данные неоднородны.

### Лемматизация

Выделим леммы в значениях столбца с целями получения кредита, осуществим предпросмотр:

In [92]:
m = Mystem()
df['purpose'] = df['purpose'].apply(m.lemmatize)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,"[сыграть, , свадьба, \n]"


**Вывод**
- лемматизация данных по столбцу purpose выполнена успешно;
- для технического исполнения применена библиотека `Mystem` в сочетании с методом apply ко всему столбцу.

### Категоризация данных

Для категоризации данных создадим несколько таблиц-словарей:
- `количество детей / долг` - поскольку проверку такого условия требует наше исследование в первую очередь;
- `семейное положение / долг` - если связи между количеством детей и наличием долгов подтвердится, то данный отчет может косвенно подтвердить ее, т.к. вероятно наличие детей / большего количества детей в семьях;
- `уровень дохода / долг` - такая связь видится разумной в первую очередь;
- `цели кредита / долг` - позволит увидеть необычные, неожиданные связи, если они есть.

Словарь "количество детей / долг":

In [93]:
children_dict = df[['children', 'debt']]

Сгруппируем данные и осуществим предпросмотр. Поскольку значения выражены при помощи 1 и 0, будем суммировать:

In [94]:
print(children_dict.groupby('children')['debt'].sum())

children
0     1063
1      444
2      194
3       27
4        4
5        0
20       8
Name: debt, dtype: int64


Из таблицы видна такая взаимосвязь: `чем меньше детей, тем чаще присутствует долг по кредиту. Чем больше детей - тем меньше клиентов с долгами`. Выбиваются из категорий представители с огромным количеством детей (20). Но это слишком нетипичное значение для отслеживания взаимосвязей.

Создадим словарь для отслеживания зависимости между семейным положением и возвратом кредита в срок:

In [95]:
family_status_dict = df[['family_status', 'debt']]
print(family_status_dict.groupby('family_status')['debt'].sum())

family_status
в разводе                 85
вдовец / вдова            63
гражданский брак         388
женат / замужем          930
не женат / не замужем    274
Name: debt, dtype: int64


Согласно данным, большее количество долгов у женатых пар. Отсюда можно сделать предварительный вывод, что наибольшее количество долгов - у <b>женатых, но бездетных<b> пар.

Далее создадим словарь отношения уровня дохода к количеству долгов. Заявщиков необходимо разделить на уровни дохода. Для градации можно использовать расчетные min, max, mean, но предварительно видно, что уровни дохода различаются очень сильно и три уровня будет недостаточно. 

Возьмем за начальный уровень МРОТ, затем до 45 тысяч официальные данные из статистики по доходам населения РФ `(https://ru.wikipedia.org/wiki/Доходы_населения_России)`. 

Ну и поскольку этих данных все равно недостаточно (метод max() показал верхнее значение в 2265604 рублей), добавим искусственные группы < 100000, < 500000 и выше 500000.

Далее напишем функцию для автоматической группировки по уровню дохода на основании полученных категорий и выполним группировку:

In [96]:
# Функция для распределения по уровню дохода:
def sort_income(income):
    if income < 11129:
        return 11129
    if income < 27000:
        return 27000
    if income < 45000:
        return 45000
    if income < 100000:
        return 100000
    if income < 500000:
        return 500000
    else:
        return '500000+'
    
income_dict = df[['total_income', 'debt']]
income_dict['income_level'] = income_dict['total_income'].apply(sort_income)
income_dict_grouped = income_dict.groupby('income_level')['debt'].sum()
print(income_dict_grouped)

income_level
27000         1
45000        13
100000      340
500000     1372
500000+      14
Name: debt, dtype: int64


Из распределения видно, что наибольшее количество должников `среди богатых (но не сверхбогатых!)` людей.

Создадим словарь "цели кредита / долг". Для этого используем полученные ранее результаты лемматизации. 

Нам необходимо выделить однословные категории и избавиться от "мусора". Сначала проверим данные при помощи метода Counter:

In [97]:
from collections import Counter
print(df['purpose'].apply(Counter).sum())

Counter({' ': 33609, '\n': 21478, 'недвижимость': 6351, 'покупка': 5899, 'жилье': 4465, 'автомобиль': 4304, 'образование': 4012, 'с': 2919, 'операция': 2603, 'свадьба': 2346, 'на': 2229, 'свой': 2228, 'строительство': 1877, 'высокий': 1374, 'коммерческий': 1314, 'получение': 1314, 'для': 1293, 'жилой': 1228, 'сделка': 943, 'заниматься': 908, 'дополнительный': 906, 'проведение': 776, 'сыграть': 773, 'сдача': 652, 'семья': 641, 'собственный': 634, 'со': 627, 'ремонт': 611, 'подержанный': 488, 'подержать': 478, 'приобретение': 460, 'профильный': 434})


Заменяем наборы слов на название категории из одного слова в словаре `"цели кредита / долг"`. Вручную отобрали следующие категории: `недвижимость, жилье, автомобиль, образование, свадьба, строительство, ремонт`. 

Из списка видно, что по количеству кредитов безусловно лидирует `"недвижимость", "жилье", "автомобиль"`.

Почти всякое жилье является недвижимостью (но не всякая недвижимость - жилье). Объединим эти две категории, а также "операция (с недвижимостью)" в одну - "недвижимость".

In [98]:
df_purpose = df[['purpose', 'debt']]
df_purpose.loc[df_purpose['purpose'].apply(str).str.contains('недвижимость'), 'purpose'] = 'недвижимость'
df_purpose.loc[df_purpose['purpose'].apply(str).str.contains('жилье'), 'purpose'] = 'недвижимость'
df_purpose.loc[df_purpose['purpose'].apply(str).str.contains('автомобиль'), 'purpose'] = 'автомобиль'
df_purpose.loc[df_purpose['purpose'].apply(str).str.contains('образование'), 'purpose'] = 'образование'
df_purpose.loc[df_purpose['purpose'].apply(str).str.contains('операция'), 'purpose'] = 'недвижимость'
df_purpose.loc[df_purpose['purpose'].apply(str).str.contains('свадьба'), 'purpose'] = 'свадьба'
df_purpose.loc[df_purpose['purpose'].apply(str).str.contains('строительство'), 'purpose'] = 'строительство'
df_purpose.loc[df_purpose['purpose'].apply(str).str.contains('ремонт'), 'purpose'] = 'ремонт'

Проверим, все ли категории были заменены:

In [99]:
print(df_purpose['purpose'].unique())

['недвижимость' 'автомобиль' 'образование' 'свадьба']


Сгруппируем данные по цели кредита:

In [100]:
df_purpose_grouped = df_purpose.groupby('purpose')['debt'].sum()
print(df_purpose_grouped)

purpose
автомобиль      402
недвижимость    782
образование     370
свадьба         186
Name: debt, dtype: int64


Создадим сводные таблицы для наглядного отображения всех зависимостей.

**Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Зависимость есть. Заметно, что `чаще возвращают кредиты бездетные либо крайне многодетные заявщики`.

In [101]:
children_grouped = children_dict.groupby(['children']).agg({'debt':'mean'})
print(children_grouped)

              debt
children          
0         0.075129
1         0.092154
2         0.094404
3         0.081818
4         0.097561
5         0.000000
20        0.105263


**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Некоторая зависимость есть. Заметно, что `чаще возвращают кредиты как вдовцы/вдовы, так и люди в разводе`. `Количество невозвратных кредитов выше у неженатых пар и пар в гражданском браке`.

In [102]:
family_grouped = family_status_dict.groupby(['family_status']).agg({'debt':'mean'})
print(family_grouped)

                           debt
family_status                  
в разводе              0.071369
вдовец / вдова         0.065900
гражданский брак       0.093001
женат / замужем        0.075298
не женат / не замужем  0.097578


**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Да, такая зависимость есть. Заметно, что `чаще возвращают кредиты люди со средним достатком и сверхобеспеченные люди`. `Лидируют по невозвратам кредиты для людей с низким уровнем дохода`. `Люди со средним достатком - самые дисциплинированные`.

In [103]:
income_grouped = income_dict.groupby(['income_level']).agg({'debt':'mean'})
print(income_grouped)

                  debt
income_level          
27000         0.083333
45000         0.063725
100000        0.080208
500000        0.081662
500000+       0.063063


**Как разные цели кредита влияют на его возврат в срок?**

Сводная таблица по целям кредита. Заметно, что `чаще не возвращают кредиты, взятые на покупку недвижимости и автомобилей`:

In [104]:
purpose_grouped = df_purpose_grouped.groupby(['purpose']).agg({'debt':'mean'})
print(purpose_grouped)

              debt
purpose           
автомобиль     402
недвижимость   782
образование    370
свадьба        186


**Вывод:**
- В лидерах по количеству долгов - `кредиты на недвижимость и автомобили`. Интересно отметить, что кредитов на свадьбу возвращают больше (не хотят входить в семейную жизнь с долгами?);
- Также, видится `вероятная связь между категориями и уровнями дохода`, т.е. вероятней всего, кредиты на авто и недвижимость берут состоятельные люди, и у них же самый большой процент невозвратов;
- Связь между наличием детей и возвратами кредитов есть, но она нелинейная: однозначно можно сказать, что кредиты чаще возвращают бездетные заявщики либо заявщики с 5 детьми, с другим количеством детей процент возврата плюс-минус одинаков.

## Общий вывод

При работе с данными исследования были обнаружены несоответствия в данных, такие как: неверный формат по данным трудового стажа, отсутствующие данные по трудовому стажу и уровню дохода. Недостающие данные по уровню трудового дохода я заменил средне-медианными по таким же категориям. Строки с отсутствующими данными по трудовому стажу я заменил нулевыми значениями. 
    
При работе были исправлены ошибки в форматировании, обработан и стандартизирован формат текстовых данных. 
    
Далее, по категориям заявщиков были прослежены зависимости, выводы подтверждены сгруппированными данными в табличном виде.
    
`Общий вывод видится таким: чаще всего задерживаются кредиты у людей с невысоким достатком и с детьми, лидирует по процентам невозврата кредит на недвижимость и автомобили. Бездетные заявщики самые дисциплинированные`.